In [44]:
# import libraries
# pyodbc for ODBC connection
import pyodbc
# pandas for 'excel' like sheets
import pandas as pd
# datetime for date computer date

# if you do not use windows authentication, use the following instead:
# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

#SQL Query pulls into Scottsdale arrestees with AZ arrestees already excluded 
ArresteesbyState = '''
 select  
	SUBSTRING(
		[City of Arrestee]
		,charindex(',',[City of Arrestee]) +2
		,2
	) as State_of_Arrestee
	,count(*) as _Count_
from [dbo].[spd_PDArrests$]
where [City of Arrestee] not like '%, AZ %' and --AZ arrestees excluded
	[City of Arrestee] like '%, __ _____' --messy, non-standard records excluded
group by SUBSTRING(
		[City of Arrestee]
		,charindex(',',[City of Arrestee]) +2
		,2
	)
  
  '''

# save the sql query to a pandas dataframe
ArresteesbyState = pd.read_sql(ArresteesbyState, connection)
connection.close()

ArresteesbyState.sort_values(['_Count_'],ascending=[0]).head(10)

# ArresteesbyState._Count_.sum()

#Create column 'Percent'
ArresteesbyState['Percent'] = ArresteesbyState._Count_.apply(lambda x: x/ArresteesbyState._Count_.sum())

#format with more decimal places
pd.options.display.float_format = '{:,.5f}'.format

ArresteesbyState['Percent'] = ArresteesbyState.Percent.astype(float)

#why can't I seem to get a percent here?
ArresteesbyState.head()


,State_of_Arrestee,_Count_,Percent
0,AB,5,0.00000
1,AK,1,0.00000
2,AL,4,0.00000
3,AR,1,0.00000
4,BC,5,0.00000
